In [ ]:
!pip install azureml
!pip install azureml-core
!pip install azureml-train
!pip install azureml-Dataset-runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.2/780.2 kB 56.0 MB/s eta 0:00:00
   

In [ ]:
from azureml.core import Workspace, Dataset, Experiment

#Importing Required Modules

In [ ]:
print('Accessing the workspace....')
ws                = Workspace.from_config("/content/config (3).json")

#Accessing the workspace and Authentication

Accessing the workspace....
Performing interactive authentication. Please follow the instructions on the terminal.


Interactive authentication successfully completed.


In [ ]:
print('Accessing/Creating the experiment...')
experiment = Experiment(workspace=ws, name='classify')

#creating the experiment in azure ml

Accessing/Creating the experiment...


In [ ]:
print('Start Experiment using Start Logging method...')
new_run = experiment.start_logging()

#New run in Azureml

Start Experiment using Start Logging method...


In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

#Importing required modules for model Training

In [ ]:
x,y =load_diabetes(return_X_y=True)
model=Ridge().fit(x,y)

#Loading Toy dataset

In [ ]:
y=df[['diabetes']]


In [ ]:
X=df.drop('diabetes',axis=1)

In [ ]:
X=pd.get_dummies(X)

#one hot encoding the categorical columns

In [ ]:
train_enc_cols = X.columns


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 2: Prepare your data
# X represents the features, and y represents the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create an instance of the Random Forest Classifier
clf = RandomForestClassifier()

# Step 4: Train the model
clf.fit(X_train, y_train)

# Step 5: Predict using the trained model
y_pred = clf.predict(X_test)

# Step 6: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

<ipython-input-14-1ee9387aece6>:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


Accuracy: 0.96985


In [ ]:
new_run.log("accuracy",accuracy)

#Logging the run in experiment

In [ ]:
new_run.complete()

In [ ]:
import joblib
model_file = '/content/model_clf.pkl'
joblib.dump(value=[train_enc_cols,clf],filename=model_file)

#making a pickle file for the model

['/content/model_clf.pkl']

In [ ]:
from azureml.core import Model

In [ ]:
model = Model.register(workspace=ws,model_name='Diabetes_model',
                       model_path='/content/model_clf.pkl',
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='1',
                       description='regression model for diabetes prediction')

#Registering the model in Azure ml studio

Registering model Diabetes_model


In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

In [ ]:
Diabetes_env = Environment.get(workspace=ws, name='Diabetes_env')

#Defining  a environment 

In [ ]:
Diabetes_env = Environment(name='Diabetes_env')

# Define the conda and pip dependencies
conda_deps = ['pip==20.2.4']
pip_deps = ['azureml-defaults', 'inference-schema[numpy-suport]', 'numpy', 'scikit-learn==0.22.1', 'scipy']

# Set the dependencies in the environment
Diabetes_env.python.conda_dependencies = CondaDependencies.create(conda_packages=conda_deps, pip_packages=pip_deps)

# Register the environment
Diabetes_env.register(workspace=ws)

{
    "assetId": "azureml://locations/eastus/workspaces/baa12598-6763-4066-ba2b-c1c6ff29bfdb/environments/Diabetes_env/versions/2",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230509.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "Diabetes_env",
    "pytho

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

cluster_name = 'bon'

if cluster_name not in ws.compute_targets:
    aks_config = AksCompute.provisioning_configuration(location='eastus',
                                                       vm_size='STANDARD_D11_V2',
                                                       agent_count=1,
                                                       cluster_purpose='DevTest')


    production_cluster = ComputeTarget.create(ws, cluster_name, aks_config)
    production_cluster.wait_for_completion(show_output=True)
else:
    production_cluster = ws.compute_targets[cluster_name]


#Allocating a compute target if there is none    

In [ ]:
from azureml.core.model import InferenceConfig

In [ ]:
inference_config = InferenceConfig(entry_script='entry.py',
                                   source_directory='.',
                                   environment = Diabetes_env)

#Defining a inference config

In [ ]:
namespace_name='endpoint'
version_name= 'version1'

In [ ]:
from azureml.core.webservice import AksEndpoint

In [ ]:
endpoint_deployment_config = AksEndpoint.deploy_configuration(tags={'modelVersion':'First','department':'Medical'},
                                                              description='new version',
                                                              namespace=namespace_name,
                                                              version_name=version_name,
                                                              traffic_percentile=10)

#Providing the deployement configuration

In [ ]:
compute= ComputeTarget(workspace=ws,name='bon')
endpoint_name='endpoint'

In [ ]:
endpoint= Model.deploy(workspace=ws,
                       name=endpoint_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=endpoint_deployment_config,
                       deployment_target=compute)
endpoint.wait_for_deployment(True)

#Deploying The model in a real time end point in azure